# Predicting dynamics of glass particles from structure using Machine Learning



## Introduction 

<a name = "Fig-1"></a>

<div id = "Fig-1" style="float: right; margin-left: 20px; width: 500px; text-align: center;">
    <img src="images/image.png" alt="Description" title="Dynamical Heteroginity" style="width: 400px;">
    <br>
    <span style="font-size: 16px; color: K;">Figure 1: Propensity map</span>
   
</div>
A material is said to be ‘‘glassy’’ when its typical relaxation time scale becomes of the order of, and often much larger than, the typical duration of an experiment or a numerical simulation. Glass-forming liquids exhibit a significant slowing down indynamics as the temperature decreases. These glass forming liquids shows heterogensous spatial patterns of dynamics composed of mobile and immobile domains, called dynamical heterogeneity, which is another hallmark of glassy dynamics. The magnitude of dynamical heterogeneity increases with decreasing temperature, which implies the emergence of highly non-trivial correlations at lower temperatures.

One can read more about the glass transition and dynamical heteroginity here: https://journals.aps.org/rmp/abstract/10.1103/RevModPhys.83.587. 

One can notice the Mobile (red) and immobile (blue) region in the image as shwon in the [right](#Dynamical-Heteroginity) at relaxation timescale ($\tau_\alpha$). 

Such heterogeneous dynamical patterns are statistically reproducible if one looks at the mobility or dynamic propensity of each particle obtained from an ensemble of many independent trajectories starting from a given static configuration, called the iso-configurational ensemble. Such numerical observations suggest the existence
of a relationship between the initial structure and future dynamics. 

A lot of machine learning techinques are applied to reveal the structure-dynamics relationship from another angle. Both supervised learning (where both structural and dynamical data is used in training) and unsupervised learning (where we use only structural data to train the model) methods are used to forecast the dynamics of unseen configuration. 

A few of recent work can be found here: 

- https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.130.238202
- https://arxiv.org/pdf/2311.14752
- https://pubs.aip.org/aip/jcp/article/156/20/204503/2841439
- https://pubs.aip.org/aip/jcp/article/157/20/204503/2842313
- https://www.frontiersin.org/journals/physics/articles/10.3389/fphy.2022.1007861/full

In this document we are going to apply some simple machine learning techniques like Linear regression and low level neural network to predict the dynamics of glass particles. We used Monte-Carlo simulation with 3 particles and are studying the dynamics at $\tau_\alpha$. This document provide basic toolkit to get start with this problem of studying stuructural-dynamics relationship of glassy particles (or similar system) using machine learning. 








## Propensity

<b name = "Iso-configurational"></b>
<div id = "Fig-2" style="float: right; margin-left: 20px; width: 500px; text-align: center;">
    <img src="images/image1.png" alt="Description" title="iso-configurational ensemble" style="width: 400px;">
    <br>
    <span style="font-size: 16px; color: K;">Figure 2: iso-configurational ensemble</span>
    <br>
    <span style="font-size: 12px; color: #555;"> <a href="https://gmshajahan.wixsite.com/website/single-post/2023/01/01/theory-of-the-isoconfigurational-ensemble" target="_blank">Source</a></span>
</div>

To see heterogeneous dynamics in real space and assess the influence of static structure, we use the iso-configurational ensemble. The isoconfigurational ensemble is a set of different motion trajectories of the same initial structure, but with different initial momenta. (Different random seed for Monte-Carlo).

In particular, we compute a propensity parameter, $p_i$, which is given by

$$
    p_i(t) = \left\langle |\Delta {\bf r}^{\rm CR}_i(t)| \right\rangle_{\rm iso}, 
$$

where $\langle \cdot \rangle_{\rm iso}$ is average over different trajectories starting from a given static configuration. When we plot this propensity map on the initial position of particles, we get map as shown in [Fig-1](#Fig-1). Regions with larger and smaller values of $p_i$ correspond to mobile and immobile regions. 

We also consider an another propensity parameter $q_i(t)$ based on the bond-breaking approach~\cite{yamamoto1998dynamics,shiba2012relationship}.
 $q_i(t)$ is defined by
$$
    q_i(t) = \left\langle n_i(t)/n_i(0) \right\rangle_{\rm iso} ,
$$
where $n_i(0)$ is the
number of neighbor particles within a cutoff radius $1.4\sigma_{\alpha \beta}$
of the particle $i$ and $n_i(t)$ is the number of particles that were initially part of the neighbors (at $t=0$) and are still inside a cutoff $1.8\sigma_{\alpha \beta}$ at time $t$.

These two can serve as target functions (Y) for our machine learning models. In this document we are mainly going to focus on propensity parameter $p_i(t)$ as our target function but one can also use $q_i(t)$. 

\$$
{\bf Y}_i = p_i(\tau_\alpha)
$$



## Structural Descripotrs 

We employ the Behler-Parrinello (BP) structure descriptors (https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.98.146401), which has been widely used in various problems including the prediction of glassy dynamics in two-dimensions (https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.114.108001).
The BP descriptors composed of the radial descriptor $G$ and angular descriptor $\Psi$. 

We define sets of particles, S, M, and L, which contain only small (S), medium (M), and large (L) particles, respectively. We also introduce labels $A, B$ (= S, M, L) to denote species for the particles $j$ and $k$, respectively.
{\bf In our codes, S is species=1, M is species=2, and L is species=3.}

The radial descriptor $G_i^A$ for each particle $i$ is defined by
$$
    G_i^A = {\sum_{j \in A}}' e^{-(r_{ij}-\mu)^2/\ell^2} f_c(r_{ij}) ,
$$
where $r_{ij}=|{\bf r}_i-{\bf r}_j|$ is the distance between two particles ($i$ and $j$), and $\mu$ and $\ell$ are parameters.
$\sum'$ denotes that the particle $i$ is removed from the summation.
$f_c(r)$ is the cut-off function, which is defined by $f_c(r)=\frac{1}{2}\left[ \cos(\pi r/R_c) + 1\right]$ for $r \leq R_c$ and $f_c(r)=0$ for $r>R_c$~\cite{behler2015constructing}. $R_c$ is a cut-off radius and we set $R_c=5.0 \sigma_{\rm LS}$.
We vary $\mu$ between $0.3 \sigma_{\rm LS}$ and $5.0 \sigma_{\rm LS}$ in increments of $0.1\sigma_{\rm LS}$ with $\ell=0.1\sigma_{\rm LS}$.
Thus, for each particle, we have $144(=3 \times 48)$ different functions parameterized by $\mu$ (with a fixed $\ell$).
We note that the summation runs over only the particles of a specific species $A$(=S, M, L).

The angular descriptor $\Psi_i^{AB}$ is defined by
$$
\Psi_i^{AB} = 2^{1-\zeta} {\sum_{\substack{{j \in A}, \ {k \in B}
(j \neq k)}}}' e^{-(r_{ij}^2+r_{ik}^2+r_{jk}^2)/\xi^2} \nonumber 
\times (1+\lambda \cos \theta_{ijk})^\zeta \times f_c(r_{ij}) f_c(r_{ik}) f_c(r_{jk}) ,
$$
where $\theta_{ijk}$ is the angle at the corner $i$ of the triangle defined by particles $i$, $j$, and $k$, and $\xi$, $\lambda$, and $\zeta$ are parameters varied systematically.
We employ $22$ sets of the parameters ($\xi$, $\lambda$, $\zeta$) (reported in https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.114.108001) in unit of $\sigma_{\rm LS}$.
Thus we have $132 (=6 \times 22)$ functions.

Thus, in total, we have $M=276(=144+132)$ functions for each particle.
These functions constitute a feature vector for a particle $i$, given by
$$
    {\bf X}_i=\left(X_i^{(1)}, X_i^{(2)}, \cdots, X_i^{(M)}\right) .
$$
This is the input for training model in Linear Regression and Neural Network.

In particular, we will store different structural descriptors in ${\bf X}_i$ as follows: 

$$
    {\bf X}_i=\left( G_i^{\rm S}, G_i^{\rm M}, G_i^{\rm L}, \Psi_i^{\rm SS}, \Psi_i^{\rm SM}, \Psi_i^{\rm SL}, \Psi_i^{\rm MM}, \Psi_i^{\rm ML}, \Psi_i^{\rm LL} \right) .
$$


## Summery of the data

We calculated the 30 trajectories of iso-configurational ensemble. We calculated 
$$
    p_i(t) = \left\langle |\Delta {\bf r}^{\rm CR}_i(t)| \right\rangle_{\rm iso}, 
$$

$$
    q_i(t) = \left\langle n_i(t)/n_i(0) \right\rangle_{\rm iso} ,
$$

where $\langle \cdot \rangle_{\rm iso}$ is average over these 30 trajectories starting from a given static configuration. 

We used 10 such configurtions containing 4000 particles in each configuration. Out of those 4000, 1760 were small type, 800 were medium type, and 1440 were large type. 

The data formate in the files is as following: 
$$
    {\bf R}_i=\left(x,y,type, p_i(\tau_\alpha),q_i(\tau_\alpha), G_i^{\rm S}, G_i^{\rm M}, G_i^{\rm L}, \Psi_i^{\rm SS}, \Psi_i^{\rm SM}, \Psi_i^{\rm SL}, \Psi_i^{\rm MM}, \Psi_i^{\rm ML}, \Psi_i^{\rm LL} \right) .
$$

where, $x$ is x-coordinate, $y$ is y-coordinate , $type$ is particle type (1 for small, 2 for medium, 3 for large) , and rest of definations are same. So, each row is a vector of length 281, with first five as $x,y,type,p_i(\tau_\alpha),q_i(\tau_\alpha)$ , and remaining 276 as ${\bf X}_i$ given above to be used as features for training. We used the same sequence in indexing as given above. 

Now we have 10 such configurations. So we will use 9 configurations for training and 1 unseen configuration for testing. (One can also use some for validation for neural networks). 
We will do it iteratively, like take 1 as test and 9 as training and do it for each configuration so basically 10 times for better statistics. 


#### Importing the Necessary Libraries

In [17]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from scipy.stats import pearsonr
from sklearn.neural_network import MLPRegressor

The code given below is to open a file. You can change the path and also structure of this function as per your need

In [3]:
def data_matrix_full_diff_species(ps,temperature,timescale,cnf):  
    with open(f"outputifile(mobility_map_FL2D_C3_Ps{ps}_N4000_Nalpha800_T{temperature}00_delta_max0.120_cal_stop{timescale}_Cnf{cnf},index).txt", 'r') as file:
        first_line = file.readline().strip() # read the first line of the file
        num_particles, box_length = map(float, first_line.split()[:2]) # first two numbers in the first line of the file which is information about Box Length and Num_particles
        data_matrix = np.loadtxt(file, dtype=float) # read the rest of the file into a numpy array
    return data_matrix

We are going to import all 10 configuration as one pandas file with dimension 40000X281. We are using DataFrame as it's easy to do the analysis particle wise or more specific statistics. Since it's an introductory toolkit, we will not train particle wise, but in the community that is a standard way to follow. 

In [4]:
for cnf in range(1,11):
    if cnf == 1:
        data_ =data_matrix_full_diff_species("0.00","0.30",4000000,cnf)
        data = pd.DataFrame(data_)
    else:
        data_ =data_matrix_full_diff_species("0.00","0.30",4000000,cnf)
        data = pd.concat([data,pd.DataFrame(data_)],axis=0)

data.reset_index(drop=True,inplace=True) # reset the index of the dataframe after concatenation so that each configuration is easy to call. 

## Scaling (Normalization)

The goal of scaling is to transform features to be on a similar scale. For example, consider the following two features:

Feature X spans the range 1500 to 2000000.

Feature Y spans the range 5 to 15.

<b name = "Standard-Scaling"></b>
<div id = "Fig-3" style="float: right; margin-left: 20px; width: 500px; text-align: center;">
    <img src="images/image2.png" alt="Description" title="Before-After Standard Scaling" style="width: 400px;">
    <br>
    <span style="font-size: 16px; color: K;">Figure 3: Before and After standard normalization</span>
    <br>
    <span style="font-size: 12px; color: #555;"> <a href="https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.linkedin.com%2Fpulse%2Frole-feature-scaling-machine-learning-deepak-kumar-1c&psig=AOvVaw3OrFuGLIHbdGkQZzTmw4c0&ust=1731058296143000&source=images&cd=vfe&opi=89978449&ved=0CBQQjRxqFwoTCKDcn-f0yYkDFQAAAAAdAAAAABAa" target="_blank">Source</a></span>
</div>


These two features span very different ranges. Scaling might manipulate X and Y so that they span a similar range, perhaps 0 to 1.

Also, uou can think of examples like different features are in different units and the machine know nothing about them.
**You want to normalize the features to ensure that all features contribute equally to the model**. It become more important with distance based algorithm like KNN or recently discovered [Information Imbalance](https://academic.oup.com/pnasnexus/article/1/2/pgac039/6568571).  

You can read more about importance of normalization of and various types of Normalization schemes here: https://developers.google.com/machine-learning/crash-course/numerical-data/normalization



There are many normalization methods, we are going to use what is called **Standard-Scaling** where we scale each feature such that it's mean $\mu_j$ is 0 and standard deviation $\sigma_j$ is 1. 

For this we scale each element of the feature by: 

$$
X_i^{\rm j} = (X_i^{\rm j} - \mu^{\rm j})/\sigma^{\rm j}
$$

for some feature j where $\mu^{\rm j}$ and $\sigma^{\rm j}$ are it's mean and standard deviation. 

### Some more stretigies 
We usually do not have access of test data while training our model. So we will calculate $\mu^{\rm j}$ and $\sigma^{\rm j}$ for training data only. When we get the test data we usually normalize it by using the same $\mu^{\rm j}$ and $\sigma^{\rm j}$ calculated from training data instead of calculating for it seperately and then normalizing. One reason for this can be that the training data set it quiet big as compared to test data set is more close to true population mean. 

Also, we usually normalize features and not target. 

In python we have inbuilt library for it which we will be using: 
https://scikit-learn.org/dev/modules/generated/sklearn.preprocessing.StandardScaler.html
 


In this example we are not segerating the data into training and testing data but one should segerate the training data for fitting the standard-scalar and then use this fitted model to transform both training and Test data. 

Here XX is set of 276 input features. 
scaler.fit_transform is first fitting the StandardScaler model i.e calculating $\mu^{\rm j}$ and $\sigma^{\rm j}$ and then transforming the individual data points $X_i^{\rm j}$. 
for test data we will just use scaler.transform as it is already fitted with training data. 


In [7]:

scaler = StandardScaler()
XX = data.iloc[:,5:]
YY = data.iloc[:,3]

XX_norm = scaler.fit_transform(XX)

## Linear Regression

It is a statistical method used to model the relationship between a dependent variable (response variable) and one or more independent variables (predictors). The goal is to find the best-fitting straight line (the regression line) through the data points that minimizes the sum of the squared differences between the observed values and the values predicted by the line.

In linear regression, the relationship between the dependent variable  y  and the independent variables  X  is modeled using a linear equation:

$$
 y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \cdots + \beta_p x_p + \epsilon 
$$

where, 
- $y$  is the dependent variable.
- $\beta_0$  is the y-intercept (constant term).
- $\beta_1$, $\beta_2$, $\ldots$, $\beta_p$  are the coefficients for the independent variables  $x_1$, $x_2$, $\ldots$, $x_p$ .
- $\epsilon$  is the error term, representing the difference between the observed and predicted values.

It can be written in matrix form as:

$$
 \mathbf{y} = \mathbf{X} \boldsymbol{\beta} + \boldsymbol{\epsilon} 
$$
where:

- $\mathbf{y}$ is the $n \times 1$ vector of the dependent variable.
- $\mathbf{X}$ is the $n \times (p+1)$ matrix of independent variables (including a column of ones for the intercept).
- $\boldsymbol{\beta}$ is the $(p+1) \times 1$ vector of coefficients.
- $\boldsymbol{\epsilon}$ is the $n \times 1$ vector of errors.


The coefficients can be calculated using the normal equation:

$$
 \boldsymbol{\beta} = (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y} 
$$

To know more about linear regression and check the detailed derivation of normal equation, one can look at: https://www.stat.cmu.edu/~cshalizi/mreg/15/lectures/13/lecture-13.pdf


In python we have inbuilt linear regression module: https://scikit-learn.org/1.5/modules/generated/sklearn.linear_model.LinearRegression.html
We are going to use it. One can calculate all the things like coffiecients, intercepts, mean square errors etc. (check the documentation). We are only going to use it for prediction of test data. 

## Pearson's correlation cofficient

<b name = "Pearson"></b>
<div id = "Fig-4" style="float: right; margin-left: 20px; width: 500px; text-align: center;">
    <img src="images/image3.png" alt="Description" title="Pearson's Cofficient" style="width: 400px;">
    <br>
    <span style="font-size: 16px; color: K;">Figure 4: Pearson's correlation cofficient</span>
    <br>
    <span style="font-size: 12px; color: #555;"> <a href="https://en.wikipedia.org/wiki/Pearson_correlation_coefficient" target="_blank">Source</a></span>
</div>

How to quantify our prediction ? 

There are many ways like calculating mean square error, $R^2$, etc. Pearson's cofficient it one such method which calculate how the well the prediction and ground truth values are correleted. More formally: 

[Pearson’s correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient), often denoted as  $\rho_{x,y}$ , is a measure of the linear relationship between two variables $x$ and $y$. It quantifies the degree to which two variables are linearly related. The value of  $\rho$  ranges from -1 to 1, where:

- $\rho_{x,y}$ = 1  indicates a perfect positive linear relationship,
- $\rho_{x,y}$ = -1  indicates a perfect negative linear relationship,
- $\rho_{x,y}$ = 0  indicates no linear relationship.

The formula for Pearson’s correlation coefficient is:

$$
 \rho_{x,y} = \frac{\sum (x_i - \overline{x})(y_i - \overline{y})}{\sqrt{\sum (x_i - \overline{x})^2 \sum (y_i - \overline{y})^2}} 
$$

where it is summed over all the sample points. 

## Execution: 

We are going to apply linear regression 10 times, such that one configuration is used as test data each time. 
So, the loop iteratively selecting one cnf as test and remaining as training data. 

Here is what we are doing inside the loop: 
1. Collecting the test data and saving it as X_test and y_test
2. Saving all the remaining data as training data. 
3. Fitting the StandardScaler() with the training data, and transfer the training and test data using this fitted StandardScaler().
4. Calling the LinearRegression Function and fitting it with the training data. 
5. Once fitted, it calculate the cofficients $\beta$ and slope $\beta_o$ 
6. We use this fitted model to predict the propensity of test dataset. 
7. To quantify the prediction, we calculate the pearson's cofficient. 
8. We store that in the list **pearson**. 

The **pearson** list contain pearson's cofficient for all the 10 cnfs as test data. We can average the result. 

In [24]:
pearson = []

for cnf in range(1,11):
    X_test = data.iloc[(cnf-1)*4000:cnf*4000,5:]
    y_test = data.iloc[(cnf-1)*4000:cnf*4000,3]

    X_train = data.drop(data.index[(cnf-1)*4000:cnf*4000]).iloc[:,5:]
    y_train = data.drop(data.index[(cnf-1)*4000:cnf*4000]).iloc[:,3]

    # Normalize the data
    scaler = StandardScaler()
    X_train_norm = scaler.fit_transform(X_train)
    X_test_norm = scaler.transform(X_test)

    # Train the model
    linear = LinearRegression()
    linear.fit(X_train_norm, y_train)

    # Predict the mobility
    y_pred = linear.predict(X_test_norm) # Predict the mobility
    pearson = np.append(pearson,pearsonr(y_test,y_pred)[0])
    
    
pearson = np.array(pearson)
print(f"Mean Pearson Correlation Coefficient: {pearson.mean()}")


Mean Pearson Correlation Coefficient: 0.6903249722609409


## Neural Network

<b name = "Neural-Network"></b>
<div id = "Fig-5" style="float: right; margin-left: 20px; width: 500px; text-align: center;">
    <img src="images/image4.png" alt="Description" title="Neural Network" style="width: 400px;">
    <br>
    <span style="font-size: 16px; color: K;">Figure 5: Sketch of Neural Network</span>
    <br>
    <span style="font-size: 12px; color: #555;"> <a href="https://www.google.com/url?sa=i&url=https%3A%2F%2Ftowardsdatascience.com%2Fdesigning-your-neural-networks-a5e4617027ed&psig=AOvVaw1R1bzAW5IrJYr9sXv2ZyWj&ust=1731076172695000&source=images&cd=vfe&opi=89978449&ved=0CBQQjRxqFwoTCNi4u7S3yokDFQAAAAAdAAAAABAb" target="_blank">Source</a></span>
</div>



It's a broad topic and it's impossible to cover it here in enough detail. One can follow the below material to get decently familiar with neural Networks: 
- http://neuralnetworksanddeeplearning.com/ which provide very intutive introduction about neural networks 
- Set of 5 courses by Andrew ng on Deep learning which is more on the application side: 
    - Course 1: https://youtube.com/playlist?list=PLkDaE6sCZn6Ec-XTbcX1uRg2_u4xOEky0&si=lp05oX7xLk_mxIy0
    - Course 2: https://youtube.com/playlist?list=PLkDaE6sCZn6Hn0vK8co82zjQtt3T2Nkqc&si=DnxDiNZ6SLKJi7PO
    - Course 3: https://youtube.com/playlist?list=PLkDaE6sCZn6E7jZ9sN_xHwSHOdjUxUW_b&si=O7wW7lUHEXBi7SOT
    - Course 4: https://youtube.com/playlist?list=PLkDaE6sCZn6Gl29AoE31iwdVwSG-KnDzF&si=4wfz93-VquezAfm2
    - Course 5: https://youtube.com/playlist?list=PLkDaE6sCZn6F6wUI9tvS_Gw1vaFAx6rd6&si=iH3wZcNw2GxW1itA
- A concise introduction can be found here: 
    - https://www.ibm.com/topics/neural-networks#:~:text=IBM-,What%20is%20a%20neural%20network%3F,options%20and%20arrive%20at%20conclusions.
    - https://en.wikipedia.org/wiki/Neural_network_(machine_learning)


We are going to use Multilayer Perceptron Regressor. I quick summery about it can be give as: 

Multilayer Perceptron (MLP) Regression is a type of neural network model used for regression tasks, where the goal is to predict a continuous output variable. Unlike linear regression, which models the relationship between input and output using a linear function, MLP regression can model complex, non-linear relationships

An MLP consists of multiple layers of neurons, each layer connected to the next through weighted connections. The typical structure of an MLP includes an input layer, one or more hidden layers, and an output layer. Each neuron in a layer applies a weighted sum of its inputs, passes the result through an activation function, and forwards the output to the next layer. 

It consist of: 
1. Input Layer: Takes the input features  $\mathbf{x} = [x_1, x_2, \ldots, x_n]$ .
2.	Hidden Layers: One or more layers of neurons that perform non-linear transformations of the inputs.
3.	Output Layer: Produces the final prediction.

We are gping to use MLP Regressor of scikit-learn: https://scikit-learn.org/dev/modules/generated/sklearn.neural_network.MLPRegressor.html 

    

## Hyper paramete tuning

Hyperparameters are configuration settings used to control the learning process of neural networks. Unlike model parameters (such as weights and biases) which are learned during training, hyperparameters are set before the training process begins. Examples of hyperparameters include the learning rate, number of hidden layers, number of neurons per layer, activation functions, batch size, number of epochs, and regularization techniques like dropout rates. These settings significantly influence the performance and efficiency of the neural network.

Hyperparameter tuning is the process of finding the optimal set of hyperparameters that yield the best performance for a neural network on a given task. Proper tuning is crucial because the choice of hyperparameters can drastically affect the model’s ability to learn and generalize from the data. Poorly chosen hyperparameters may lead to issues such as underfitting, where the model is too simple to capture the underlying patterns, or overfitting, where the model learns the noise in the training data and performs poorly on unseen data.

Here we are going to use GridSearchCV to search Hyper-parameter by giving a grid space. It's an inbuilt module in python and you can check it's documentation here: https://scikit-learn.org/dev/modules/generated/sklearn.model_selection.GridSearchCV.html

A brief summery of Hyper-parameter tuning can be found here: https://www.run.ai/guides/hyperparameter-tuning

In [18]:
"""
We are using the whole data set because we are going to do this experiment 10 times, so doing the GridSearchCV 10 times would be computationally expensive. 
But one should use only training data for GridSearchCV.
Here is a small set of hyperparameters to search over. But in reality, one should search over a larger set of hyperparameters.
"""

# Define the MLPRegressor
mlp = MLPRegressor(max_iter=10000 , early_stopping = True , random_state=42)

# Define the hyperparameter grid to search over
param_grid = {
    'hidden_layer_sizes': [(2),(2,2), (2,2,2)],
    'activation': ['relu'],
    'solver': ['adam'],
    'learning_rate': ['constant'],
    'alpha' : [0.01,0.1,1,10],
    "batch_size" : [50]
}
np.random.seed(42)
# Define the grid search
grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, cv=10, n_jobs=-1, verbose=2)

# Fit the grid search
grid_search.fit(XX_norm, YY)

# Return the best estimator
best_mlp = grid_search.best_estimator_
print(best_mlp)

Fitting 10 folds for each of 12 candidates, totalling 120 fits
[CV] END activation=relu, alpha=0.01, batch_size=50, hidden_layer_sizes=2, learning_rate=constant, solver=adam; total time=   5.7s
[CV] END activation=relu, alpha=0.01, batch_size=50, hidden_layer_sizes=2, learning_rate=constant, solver=adam; total time=   8.1s
[CV] END activation=relu, alpha=0.01, batch_size=50, hidden_layer_sizes=2, learning_rate=constant, solver=adam; total time=  11.5s
[CV] END activation=relu, alpha=0.01, batch_size=50, hidden_layer_sizes=2, learning_rate=constant, solver=adam; total time=  11.7s
[CV] END activation=relu, alpha=0.01, batch_size=50, hidden_layer_sizes=2, learning_rate=constant, solver=adam; total time=  11.8s
[CV] END activation=relu, alpha=0.01, batch_size=50, hidden_layer_sizes=2, learning_rate=constant, solver=adam; total time=  12.1s
[CV] END activation=relu, alpha=0.01, batch_size=50, hidden_layer_sizes=2, learning_rate=constant, solver=adam; total time=  16.4s
[CV] END activation=

We can use this best architecture we get in our final model to train and test

In [25]:
"""
Doing the same steps as we did in Linear regression just with neural network this time
"""

pearson = []

for cnf in range(1,11):
    X_test = data.iloc[(cnf-1)*4000:cnf*4000,5:]
    y_test = data.iloc[(cnf-1)*4000:cnf*4000,3]

    X_train = data.drop(data.index[(cnf-1)*4000:cnf*4000]).iloc[:,5:]
    y_train = data.drop(data.index[(cnf-1)*4000:cnf*4000]).iloc[:,3]

    # Normalize the data
    scaler = StandardScaler()
    X_train_norm = scaler.fit_transform(X_train)
    X_test_norm = scaler.transform(X_test)

    # Train the model
    #Adjust the parameters according to the best estimator
    mlp = MLPRegressor(hidden_layer_sizes=(2,2), activation='relu', solver='adam', learning_rate='constant', alpha=0.01, batch_size=50, max_iter=10000, early_stopping=True, random_state=42)
    mlp.fit(X_train_norm, y_train)

    # Predict the mobility
    y_pred = mlp.predict(X_test_norm) # Predict the mobility
    pearson = np.append(pearson,pearsonr(y_test,y_pred)[0])

pearson = np.array(pearson)
print(f"Mean Pearson Correlation Coefficient: {pearson.mean()}")

Mean Pearson Correlation Coefficient: 0.6795204415023716
